In [11]:
#from dotenv import load_dotenv
#load_dotenv("api.env")  # 👈 specify file name


In [12]:
#import os
#from dotenv import load_dotenv

#load_dotenv("api.env")  # or just load_dotenv() if renamed

#print(os.getenv("GOOGLE_API_KEY"))


In [14]:
from typing import List, TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

# Load API key
load_dotenv("api.env")
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found in api.env")

print(" API key loaded!")

class ChatState(TypedDict):
    messages: List[str]

def merge_messages(old: ChatState, new: ChatState) -> ChatState:
    return {"messages": old["messages"] + new["messages"]}

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=api_key
)

def chatbot_node(state: ChatState) -> ChatState:
    user_input = state["messages"][-1]
    response = model.invoke(user_input)

    if isinstance(response, str):
        reply_text = response
    elif hasattr(response, "content"):
        # Try to extract text safely
        reply_text = (
            response.content[0].text
            if isinstance(response.content, list) and len(response.content) > 0
            else str(response.content)
        )
    else:
        reply_text = str(response)

    return {"messages": state["messages"] + [reply_text]}

graph_builder = StateGraph(ChatState, input_schema=ChatState, output_schema=ChatState, reducer=merge_messages)
graph_builder.add_node("chatbot", chatbot_node)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

input_state = {"messages": ["What do you know about LangGraph?"]}
final_state = graph.invoke(input_state)

print("\n Chatbot Reply:")
print(final_state["messages"][-1])


 API key loaded!

 Chatbot Reply:
LangGraph is a framework for building stateful, multi-actor applications using Language Models (LLMs). It's designed to make it easier to create complex, conversational AI systems that can handle intricate workflows and interactions. Here's a breakdown of what I know about LangGraph:

**Key Concepts and Features:**

*   **State Graphs:** At the heart of LangGraph is the concept of a state graph. This graph defines the possible states your application can be in, the transitions between those states, and the actions that are triggered as the application moves from state to state. Think of it as a flowchart for your AI's behavior.
*   **Nodes:** Nodes in the graph represent individual actors or steps in the process. These nodes can be:
    *   **LLMs:** Nodes that use language models to generate text, answer questions, or perform other natural language tasks.
    *   **Tools:** Nodes that execute external functions or APIs (e.g., searching the web, access